In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from settings  import *
from preprocessing import *
from models import Relative_Positioning, StagerNet
import mne
import dataset as ds
from train import *
import decode as dec
from importlib import reload
import process 
import preprocessing as pr
reload(process)
reload(dec)
reload(ds)

<module 'dataset' from '/home/brain/EEG-ssl/dataset.py'>

In [2]:
# models params
C = 64
T =1000
M = 10
#training paramms
epochs = 20
batch_size = 40
lr = 1e-3
resume = False
#datasets params
n_train = 4000
n_test = 500
#sampling params
pos= 3000
neg = 7000

s_weights = [0.5, 1-0.5]


In [3]:
import torch
from torch.nn import *
from pylab import *
from torch import optim
from torch.utils import data
from torch import nn
from torch.nn.functional import soft_margin_loss

In [4]:
ssl_model = Relative_Positioning(StagerNet,C , T, embedding_dim = M )
ssl_model.load_state_dict(torch.load(os.path.join(ROOT, 'saved_models', 'ssl_model.pt')))
model = ssl_model.feature_extractor
subjects = SUBJECTS[:-2]
    #split data
val_subjects = [SUBJECTS[-2]]
aggregator = torch.mean
decoder = dec.Decoder(model, aggregator, C, T, embedding_dim=M, hidden_dim = 20)
decoder.to(float).to(DEVICE)


Decoder(
  (feature_extractor): StagerNet(
    (relu): ReLU()
    (spatial_conv): Conv2d(1, 64, kernel_size=(64, 1), stride=(1, 1))
    (temp_conv1): Conv2d(1, 16, kernel_size=(1, 51), stride=(1, 1), padding=(0, 25))
    (batch_norm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (temp_conv2): Conv2d(16, 16, kernel_size=(1, 51), stride=(1, 1), padding=(0, 25))
    (batch_norm2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (maxPool): MaxPool2d(kernel_size=(1, 13), stride=(1, 13), padding=0, dilation=1, ceil_mode=False)
    (flatten): Flatten()
    (dropout): Dropout(p=0.5, inplace=False)
    (linear_class): Linear(in_features=5120, out_features=10, bias=True)
  )
  (linear1): Linear(in_features=10, out_features=20, bias=True)
  (linear2): Linear(in_features=20, out_features=12, bias=True)
  (loss_fn): CrossEntropyLoss()
  (relu): ReLU()
)

In [45]:
class DecoderSampler(torch.utils.data.sampler.Sampler):
    r"""Sample des windows randomly
    Arguments:
    ---------
        dataset (Dataset): dataset to sample from
        size (int): The total number of sequences to sample
    """

    def __init__(self,dataset, batch_size,size,  weights):
    
        
        self.batch_size = batch_size
        self.size = size
        self.dataset = dataset
        self.weights = torch.DoubleTensor(weights)
        self.n_subjects = len(self.dataset.subjects)

    def __iter__(self):
        num_batches = self.size// self.batch_size
        n_subject_samples = self.batch_size //self.n_subjects
        while num_batches > 0:
            #sample a subject 
            batch_subjects = choice(self.dataset.subjects,replace = False, size =5)
            for subject in  batch_subjects:# TODO sample subjects ?
                sampled = 0
                #for each subject sample `n_subject_samples`  couples (target class, trial)
                while sampled < n_subject_samples:
                    indice  = torch.multinomial(
                self.weights, 1, replacement=True)
                    trial = choice(self.dataset.trials)
                    sampled += 1
                    yield ( indice,trial, subject)
                
            num_batches -=1

    def __len__(self):
        return len(self.size)   

class SequentialSampler(torch.utils.data.sampler.Sampler):
    r"""sampler for validation iterate sequentialy on subjects 
    Arguments:
    ---------
        dataset (Dataset): dataset to sample from
        size (int): The total number of sequences to sample
    """

    def __init__(self,dataset, batch_size,size,  weights):
    
        
        self.batch_size = batch_size
        self.size = size
        self.dataset = dataset
        self.weights = torch.DoubleTensor(weights)
        self.n_subjects = len(self.dataset.subjects)
        
    def __iter__(self):
        num_batches = self.size// self.batch_size
        n_subject_samples = self.batch_size //self.n_subjects
        while num_batches > 0:
            sampled = 0
            while sampled < self.batch_size:   
                for subject in self.dataset.subjects:# TODO sample subjects ?
                    for indice in range(len(self.weights)):
                        for trial in self.dataset.trials:                
                                sampled += 1
                                yield ( indice,trial, subject)
                
            num_batches -=1

    def __len__(self):
        return len(self.size)
class  Decoder_Dataset(torch.utils.data.Dataset):
    '''
    Classe dataset  pour les differents sampling
    '''
    def __init__(self, subjects, trials, T , step, condition = 1):

        self.subjects = subjects
        self.temp_len = T
        self.step = step
        self.trials = trials
        self.condition = condition
    def load_epoch(self, index):
        #print(index)
        ( indice,trial, subject) = index
        stim_id = STIMULUS_IDS[indice]
        
        epochs_path = os.path.join(RAW_DIR, f"{subject}-epochs_{stim_id}_{self.condition}-epo.fif")
        if subject in recording_with_mastoid_channels:
            epochs = mne.read_epochs(epochs_path, verbose = False)[trial].get_data()
        else :
            epochs = mne.read_epochs(epochs_path, verbose = False)[trial].drop_channels(['EXG5','EXG6']).get_data()

        return epochs
    def __getitem__(self, index):
        classe = STIMULUS_IDS[index[0]]
        sampled = self.load_epoch(index)
        x_sample = torch.tensor(sampled).to(DEVICE).unfold(-1, self.temp_len, self.step ).permute(2,0,1,3)
        return x_sample, index[0]
    def __len__(self): len(self.subjects)*len(self.trials )*12
        
def collate(batch):
   
    samples= [item[0] for item in batch]
    
    targets = [item[1] for item in batch]
    
    return samples, targets


In [ ]:
import numpy as np # linear algebra
import os
from settings  import *
from preprocessing import *
from models import Relative_Positioning, StagerNet, Decoder
from dataset import Decoder_Dataset, DecoderSampler
import process
from train import load_losses, save_losses
from torch import nn, optim
import argparse 



In [43]:
def train_decoder(model, train_dataset, val_dataset,samplers, n_epochs=20, lr=1e-3, batch_size=256, load_last_saved_model=False, num_workers=8):
	
	
	if load_last_saved_model:
		model.load_state_dict(torch.load(os.path.join(ROOT, SAVED_MODEL_DIR, 'decoder.pt')))

	if torch.cuda.device_count() > 1:
		model = nn.DataParallel(model)
        
	model.to(DEVICE)
    
   

	train_loader = train_loader = torch.utils.data.DataLoader(train_dataset,batch_size = batch_size, num_workers=0,
                                          sampler = samplers["train"], collate_fn = collate)
	val_loader = val_loader = torch.utils.data.DataLoader(val_dataset, num_workers=0,
                                          sampler = samplers["val"])
	new_train_losses, new_test_losses = _train_epochs_dec(model, train_loader, test_loader, 
																				 dict(epochs=n_epochs, lr=lr))

	if load_last_saved_model:
		train_losses, test_losses = load_losses(SAVED_MODEL_DIR, 'decoder')
	else:
		train_losses = []
		test_losses = []
	
	train_losses.extend(new_train_losses)
	test_losses.extend(new_test_losses)

	save_losses(train_losses, test_losses, SAVED_MODEL_DIR, 'decoder')

	return train_losses, test_losses, model



In [51]:
def _train_dec(model, train_loader, optimizer, epoch):
    
    model.train()
    train_losses = []
    batch_size = train_loader.batch_size
    for batch_x, batch_y in train_loader:
        print(batch_y)
        print('in')
        loss = 0
        for x,y in zip(batch_x, batch_y):
            x = x.squeeze(dim= 0 )
            y = y.to(DEVICE)
            out = model(x)
            #print(out.shape)
            loss = loss+ model.loss_fn(out.unsqueeze(dim =0), y)
        loss =loss/batch_size
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    return mean(train_losses)
def _eval_loss_dec(model, data_loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for x, y in data_loader:
            x = x.squeeze(dim= 0 )
            y = y.to(DEVICE)
            loss = model.loss_fn(model(x).unsqueeze(dim = 0), y)
            total_loss += loss #* x[0].shape[0] #
        avg_loss = total_loss / data_loader.sampler.size# / len(data_loader.dataset)
    return avg_loss.item()

def _train_epochs_dec(model, train_loader, test_loader, train_args):
	epochs, lr = train_args['epochs'], train_args['lr']
	optimizer = optim.Adam(model.parameters(), lr=lr)
	if not os.path.exists(SAVED_MODEL_DIR):
		os.makedirs(SAVED_MODEL_DIR)
	
	train_losses = []
	test_losses = [_eval_loss_dec(model, test_loader)]
	for epoch in range(1, epochs+1):
		model.train()
		train_losses.extend(_train_dec(model, train_loader, optimizer, epoch))
		test_loss = _eval_loss_dec(model, test_loader)
		test_losses.append(test_loss)
        y_true, y_pred = get_test_results(model, test_loader)
        
		print(f'Epoch {epoch}, Test loss {test_loss:.4f}, \tAccuracy: {100*acc_score:.2f}%')
		
		# save model every 10 epochs
		if epoch % 2 == 0:
			torch.save(model.state_dict(), os.path.join(ROOT, 'saved_models', 'decoder_epoch{}.pt'.format(epoch)))
	torch.save(model.state_dict(), os.path.join(ROOT, 'saved_models', 'decoder.pt'))
	return train_losses, test_losses

train_dataset = Decoder_Dataset(subjects,[0,1,2], T, step = 512)
val_dataset = Decoder_Dataset(subjects,[3,4], T, step = 512)
train_sampler = DecoderSampler(train_dataset, batch_size = 21, weights = [1/12]*12, size = 65)
val_sampler = SequentialSampler(val_dataset, batch_size = 60, weights = [1/12]*12, size = 65)
samplers = {"train" : train_sampler, "val": val_sampler}
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size = 21, num_workers=0,
                                          sampler = samplers["train"], collate_fn = collate)
val_loader = torch.utils.data.DataLoader(val_dataset, num_workers=0,
                                          sampler = samplers["val"])
#for x ,y in train_loader:
    #print(y)
optimizer = optim.Adam(decoder.parameters(), lr=1e-3)
#train_losses = _train_dec(decoder, train_loader,optimizer, 2 )
_eval_loss_dec(decoder, val_loader)

Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection |

8.248205721014568

In [35]:
train_losses

3.084803566683392

In [147]:
train_sampler = DecoderSampler(train_dataset, batch_size = 15, weights = [1/12]*12, size = 65)
val_sampler = SequentialSampler(val_dataset, batch_size = 60, weights = [1/12]*12, size = 65)
samplers = {"train" : train_sampler, "val": val_sampler}
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size = 15, num_workers=0,
                                          sampler = samplers["train"], collate_fn = collate)
for x ,y in train_loader:
    print(y)

Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
[tensor([5]), tensor([10]), tensor([0]), tensor([1]), tensor([11]), tensor([5]), tensor([5]), tensor([2])]


In [50]:
#import get_results as res
from sklearn.metrics import balanced_accuracy_score, accuracy_score
def get_test_results(model, test_loader):
    y_true = []
    y_pred = []
    model.eval()
    softmax = nn.Softmax(dim = 1)
    with torch.no_grad():
        
        for x, y in test_loader:
            
            x = x.to(DEVICE).to(float).contiguous().squeeze(dim= 0)
            #x = x.reshape(-1,1,x.shape[2], 1000)
            y = y.to(DEVICE).to(float).contiguous()
            out = model(x)
            _, predicted = torch.max(softmax(out.unsqueeze(dim =0)), 1)
            y_true.extend(list(y.cpu().numpy()))
            y_pred.extend(list(predicted.cpu().numpy()))
    return y_true, y_pred

def decoder_scores(model, subjects, trials):
    
    test_dataset = Decoder_Dataset(subjects,trials, T, step = 512)
    test_sampler = SequentialSampler(test_dataset, batch_size = 60, weights = [1/12]*12, size = 65)
    test_loader = torch.utils.data.DataLoader(test_dataset, num_workers=0,
                                          sampler = test_sampler)
    y_true, y_pred = get_test_results(model, test_loader)
    acc_score = accuracy_score(y_true, y_pred)
    balanced_acc_score = balanced_accuracy_score(y_true, y_pred)
    print(f'Performance of the network on the test trials:')
    print(f'\tAccuracy: {100*acc_score:.2f}%')
    print(f'\tBalanced accuracy: {100*balanced_acc_score:.2f}%')
    return acc_score, balanced_acc_score

decoder_scores(decoder, subjects, [3,4])

Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection | Average EEG reference, active : True, n_channels : 66>
Removing projector <Projection |

(0.08333333333333333, 0.08333333333333333)

In [126]:
from sklearn.metrics import balanced_accuracy_score, accuracy_score
acc_score = accuracy_score(y_true, y_pred)
balanced_acc_score = balanced_accuracy_score(y_true, y_pred)
print(acc_score,balanced_acc_score )

0.09523809523809523 0.09523809523809523


In [4]:
train_dataset = ds.Decoder_Dataset(subjects,[0,1,2], T, step = 512)
val_dataset = ds.Decoder_Dataset(subjects,[3,4], T, step = 512)
train_sampler = ds.DecoderSampler(train_dataset, batch_size = 12, weights = [1/12]*12, size = 65)
val_sampler = ds.DecoderSampler(val_dataset, batch_size = 12, weights = [1/12]*12, size = 65)
samplers = {"train" : train_sampler, "val": test_sampler}
trainloader = torch.utils.data.DataLoader(dataset, num_workers=0,sampler = samplers["train"])
train_losses, test_losses, model = train_decoder(decoder, dataset,val_dataset,sampler, n_epochs=epochs, lr=lr)

Opening raw data file /home/brain/openmiir/raw_data/P01-raw.fif...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 64)  idle
    Range : 0 ... 2478165 =      0.000 ...  4840.166 secs
Ready.
Reading 0 ... 2478165  =      0.000 ...  4840.166 secs...


In [21]:
def main(model_name : str, data : List[torch.tensor], epochs : int, C : int, T : int, M : int, batch_size : int, n_train : int, 
         n_test: int , s_weights : List[float], pos : int, neg : int, lr : float = 1e-3 ):
    #define ssl model
    X_train = data[0]
    X_test = data[1]
    ssl_model = Relative_Positioning(StagerNet,C , T, embedding_dim = M )
    ssl_model.to(float)
    # datasets
    train_dataset =  RP_Dataset(X_train, sampling_params = (pos, neg), temp_len = T ,
                                n_features = C )
    test_dataset =  RP_Dataset(X_test, sampling_params = (pos, neg), temp_len = T ,
                                n_features = C )

    train_sampler = WeightedSampler(train_dataset, batch_size = batch_size ,size = n_train,  
                              weights = s_weights)
    test_sampler = WeightedSampler(test_dataset, batch_size = batch_size ,size = n_test,  
                              weights = s_weights)
    samplers = {"train" : train_sampler, "val": test_sampler}


    #train ssl 
    train_losses, test_losses, model = train_ssl(ssl_model, train_dataset, test_dataset,
                                                 samplers,n_epochs=epochs, lr=lr,batch_size= batch_size, 
                                                 load_last_saved_model=False, num_workers= 0)

    return train_losses, test_losses, model


Epoch 1, Test loss 0.0003
Epoch 2, Test loss 0.0003
Epoch 3, Test loss 0.0003
Epoch 4, Test loss 0.0002
Epoch 5, Test loss 0.0002
Epoch 6, Test loss 0.0002
Epoch 7, Test loss 0.0002
Epoch 8, Test loss 0.0002
Epoch 9, Test loss 0.0002
Epoch 10, Test loss 0.0002
Epoch 11, Test loss 0.0002
Epoch 12, Test loss 0.0002
Epoch 13, Test loss 0.0002
Epoch 14, Test loss 0.0002
Epoch 15, Test loss 0.0002
Epoch 16, Test loss 0.0002
Epoch 17, Test loss 0.0002
Epoch 18, Test loss 0.0002
Epoch 19, Test loss 0.0002
Epoch 20, Test loss 0.0002
